In [1]:
import numpy as np

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 03 seconds to download the candles


In [3]:
mufex.exchange_settings

In [4]:
candles

,open,high,low,close
timestamp,,,,
2023-09-30 01:25:00,26914.6,26914.6,26900.6,26904.7
2023-09-30 01:30:00,26904.7,26911.0,26899.3,26903.5
2023-09-30 01:35:00,26903.5,26906.7,26901.5,26901.5
2023-09-30 01:40:00,26901.5,26901.5,26886.6,26887.6
2023-09-30 01:45:00,26887.6,26899.8,26887.6,26899.7
...,...,...,...,...
2023-10-06 23:40:00,27927.1,27927.1,27908.5,27916.7
2023-10-06 23:45:00,27916.7,27926.8,27914.0,27926.8
2023-10-06 23:50:00,27926.8,27926.8,27916.4,27916.5


In [5]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([1.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.Provided]),
    max_equity_risk_pct=np.array([4]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([150, 250]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [19]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.RegularBacktest,
    candles=candles,
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 2
Total combinations of settings to test: 8

Total candles: 2,000
Total candles to test: 16,000


In [20]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,0,0,96.0,-103.470818,28.12,-0.873573,-508.5292,491.470969
6,0,1,93.0,-104.039639,27.96,-0.881815,-509.8993,490.100848
2,1,0,85.0,-102.506266,24.71,-0.876640,-506.1883,493.811881
4,1,1,82.0,-98.671130,24.39,-0.876958,-496.6557,503.344395
1,2,0,96.0,-103.470818,28.12,-0.873573,-508.5292,491.470969
7,2,1,93.0,-104.039639,27.96,-0.881815,-509.8993,490.100848
3,3,0,85.0,-102.506266,24.71,-0.876640,-506.1883,493.811881
5,3,1,82.0,-98.671130,24.39,-0.876958,-496.6557,503.344395


In [21]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.CandleBacktest,
    candles=candles,
)
candle_backtest = backtest_df_only(
    starting_equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 4
Total order settings to test: 2
Total combinations of settings to test: 8

Total candles: 2,000
Total candles to test: 16,000


In [22]:
candle_backtest.sort_values(['ind_set_idx','or_set_idx'])

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
6,0,0,69.0,-130.297012,15.94,-0.929230,-565.7782,434.221874
0,0,1,67.0,-111.253889,16.42,-0.911602,-526.6360,473.364068
4,1,0,69.0,-130.167394,15.94,-0.925766,-565.5336,434.466403
2,1,1,67.0,-110.968137,16.42,-0.912098,-525.9949,474.005229
7,2,0,68.0,-131.544514,16.18,-0.933272,-568.1176,431.882399
3,2,1,66.0,-112.011784,16.67,-0.915119,-528.3281,471.671895
5,3,0,69.0,-127.563596,15.94,-0.929024,-560.5623,439.437598
1,3,1,67.0,-111.137508,16.42,-0.911785,-526.3751,473.624989
